Load jupyter sql extension


In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/raw/frantamarit.db')

In [11]:
sql = \
"SELECT rank_info.id, rank_info.name as fullName, rank_info.position,personal_info.lastName,\
personal_info.age, personal_info.gender, personal_info.country, personal_info.image,\
business_info.Source as source, business_info.worth, business_info.worthChange,\
business_info.realTimePosition \
FROM rank_info \
LEFT JOIN personal_info \
ON rank_info.id = personal_info.id \
LEFT JOIN business_info \
ON rank_info.id = business_info.id \
ORDER BY rank_info.position DESC"

df_master = pd.read_sql_query(sql, engine)

In [50]:
df_master['country'].sort_values().unique()

array(['Angola', 'Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil',
       'Canada', 'Chile', 'China', 'Colombia', 'Cyprus', 'Czech Republic',
       'Denmark', 'Egypt', 'Finland', 'France', 'Georgia', 'Germany',
       'Greece', 'Hong Kong', 'Iceland', 'India', 'Indonesia', 'Ireland',
       'Israel', 'Italy', 'Japan', 'Kazakhstan', 'Kuwait', 'Lebanon',
       'Liechtenstein', 'Macau', 'Malaysia', 'Mexico', 'Monaco',
       'Morocco', 'Netherlands', 'New Zealand', 'None', 'Norway', 'Oman',
       "People's Republic of China", 'Peru', 'Philippines', 'Poland',
       'Qatar', 'Russia', 'Singapore', 'South Africa', 'South Korea',
       'Spain', 'St. Kitts and Nevis', 'Swaziland', 'Sweden',
       'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'UK', 'USA',
       'Ukraine', 'United Arab Emirates', 'United Kingdom',
       'United States', 'Vietnam', None], dtype=object)

In [27]:
#To use afterwards
def lowercase_feature(df,col):
    return df[col].str.lower()

In [29]:
to_lower_cols = ['lastName', 'fullName', 'country', 'source']

for i in to_lower_cols:
    df_master[i] = lowercase_feature(df_master, i)

In [28]:
def extract_str(df, col, regex):
    return df[col].str.extract(regex)

def extract_float(df, col, regex):
    return df[col].str.extract(regex).astype('float64')

In [30]:
df_master['sourceDetails'] = extract_str(df_master,'source',r"> (.*)")

In [31]:
df_master['source'] = extract_str(df_master,'source',r"(.*)  =")

In [32]:
#In USD Billion
df_master['worth'] = extract_float(df_master,'worth',r"(\d+.?\d)")

In [33]:
#In USD Billion
df_master['worthChange'] = extract_float(df_master,'worthChange',r"(-?\d+.?\d*) ")

In [34]:
df_master['country'].replace("People's Republic of China", 'China',inplace=True)

In [47]:
df_master['country'].replace("usa", 'united stated',inplace=True)

In [46]:
df_master['country'].sort_values().unique()

array(['angola', 'argentina', 'australia', 'austria', 'belgium', 'brazil',
       'canada', 'chile', 'china', 'colombia', 'cyprus', 'czech republic',
       'denmark', 'egypt', 'finland', 'france', 'georgia', 'germany',
       'greece', 'hong kong', 'iceland', 'india', 'indonesia', 'ireland',
       'israel', 'italy', 'japan', 'kazakhstan', 'kuwait', 'lebanon',
       'liechtenstein', 'macau', 'malaysia', 'mexico', 'monaco',
       'morocco', 'netherlands', 'new zealand', 'none', 'norway', 'oman',
       "people's republic of china", 'peru', 'philippines', 'poland',
       'qatar', 'russia', 'singapore', 'south africa', 'south korea',
       'spain', 'st. kitts and nevis', 'swaziland', 'sweden',
       'switzerland', 'taiwan', 'thailand', 'turkey', 'uk', 'ukraine',
       'united arab emirates', 'united kingdom', 'united states', 'usa',
       'vietnam', None], dtype=object)

In [35]:
#Using Float but better to be treated as integer later on
df_master['age'] = extract_float(df_master,'age',r'(\d+)')

In [39]:
df_master['age'].unique()

array([  54.,   52.,   51.,   55.,   53.,   49.,   59.,   85.,   67.,
         66.,   63.,   69.,   77.,   73.,   62., 1984.,   64.,   80.,
         91.,   75.,   79.,   60.,   72.,   76.,   47.,   70.,   50.,
         56.,   38., 1986.,   88.,   48.,   74.,   45., 1990., 1992.,
         71.,   86.,   61.,   42.,   nan,   78.,   83.,   68.,   40.,
         65.,   44.,   43.,   81.,   57.,   46.,   82.,   58., 1989.,
         41.,   92., 1983., 1982.,   90., 1985., 1987., 1988.,   84.,
         94.,   39.,   87.,   96.,   89., 1996., 1998.,   95., 1991.,
         97.,   93., 1994.,   99.])

In [40]:
df_master['age'] = [2018 - i if i > 1000 else i for i in df_master['age']]

In [12]:
#1 assigned to Female, 2 to Male & 0 to nulls.
df_master['gender'] = [0 if i == 'F' or i == 'Female' else 1 if i == 'M' or i == 'Male' else '' for i in df_master['gender']]

In [15]:
df_master['gender'] = pd.to_numeric(df_master['gender'], errors='ignore')

In [17]:
df_master['gender'].mean()

0.8874695863746959

In [124]:
for i in df_master.columns:
    print(i)

id
fullName
position
lastName
age
gender
country
image
source
worth
worthChange
realTimePosition
sourceDetails


In [51]:
for i,j in zip(df_master.dtypes, df_master.columns):
    if i == 'float64' or i == 'int64':
        df_master[j].fillna(0, inplace=True)
    else:
        df_master[j].fillna('', inplace=True)

In [128]:
df_master.to_csv(f'../data/processed/cleaned_output.csv', sep=';', index=False)

In [14]:
len(df_master)

2260

In [20]:
#We don't want to get rid of the NaN yet, so we'll fill them with 0s.
def fill_0(x):
    return x.fillna(0, inplace=True)

fill_0(df_rank_info)
df_rank_info

,id,name,position
0,5542,anant asavabhokhin & family,0.0
1,4183,arash ferdowsi,0.0
2,4761,banthoon lamsam & family,0.0
3,1449,bill adderley & family,0.0
4,1624,boris zingarevich,0.0
...,...,...,...
2255,9216,zhao xiaoqiang,2204.0
2256,5887,zhou liangzhang,2205.0
2257,8536,zhu xingming,2206.0
2258,2599,zhuo jun,2207.0


In [52]:
#get nulls

def get_null(x):
    return x.isnull().sum()

In [53]:
get_null(df_master)

id                  0
fullName            0
position            0
lastName            0
age                 0
gender              0
country             0
image               0
source              0
worth               0
worthChange         0
realTimePosition    0
dtype: int64

In [119]:
for i in df_master.dtypes:
    if i == 'float64':
        print('yest')
    else:
        print('no')

no
no
yest
no
yest
no
no
no
no
yest
yest
yest
no


In [120]:
df_master.dtypes

id                    int64
fullName             object
position            float64
lastName             object
age                 float64
gender                int64
country              object
image                object
source               object
worth               float64
worthChange         float64
realTimePosition    float64
sourceDetails        object
dtype: object

In [ ]:
personal_info.head()

In [ ]:
master_info.isnull().sum()